In [1]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import seaborn as sns
from scipy.signal import butter,filtfilt,welch
from scipy import signal


In [2]:
def Regular_1(L,Inh):
    L2=L*L
    Matrix=np.zeros((L2,L2),int)
    rng=np.random.default_rng(seed=1)
    for i in range(L):
        for j in range(L):
            ia=i
            ji=j
            id=i
            jd=j
            if ia+1==L:
                ia=-1
            if id==0:
                id=L
            if jd+1==L:
                jd=-1
            if ji==0:
                ji=L
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else :
                I=1
            Matrix[i*L+j][i*L+jd+1]=1*(2*I-1)## derecha I=0 -1,I=1 1
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else:
                I=1
            Matrix[i*L+j][i*L+ji-1]=-1*(2*I-1)## izquieda
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else:
                I=1
            Matrix[i*L+j][(ia+1)*L+j]=-1*(2*I-1)## arriba 
            I=rng.uniform(0.0, 1.0)
            if I<Inh:
                I=0
            else: 
                I=1
            Matrix[i*L+j][(id-1)*L+j]=-1*(2*I-1)# Abajo
    return Matrix

def small_word_Bi(L,P,Inh):
    Matrix=Regular_1(L,Inh)
    rng=np.random.default_rng(seed=1)
    for i in range(L*L):
        for j in range(L*L):
            if Matrix[i][j]==1 and rng.uniform(0.0, 1.0)<P:
                Matrix[i][j]==0
                Matrix[j][i]==0
                jj=rng.integers(0,L*L-1)
                if jj==i or jj==j:
                    jj=rng.integers(0,L*L-1)
                    Matrix[jj][i]=2
                    Matrix[i][jj]=2
                else:
                    Matrix[jj][i]=2
                    Matrix[i][jj]=2
            elif Matrix[i][j]==-1 and rng.uniform(0.0, 1.0)<P:
                Matrix[i][j]==0
                Matrix[j][i]==0
                jj=rng.integers(0,L*L-1)
                if jj==i or jj==j:
                    jj=rng.integers(0,L*L-1)
                    Matrix[jj][i]=-2
                    Matrix[i][jj]=-2
                else:
                    Matrix[jj][i]=-2
                    Matrix[i][jj]=-2
    for i in range(L*L):
        for j in range(L*L):
            if Matrix[i][j]==2:
                Matrix[i][j]=1
                Matrix[j][i]=1
            elif Matrix[i][j]==-2:
                Matrix[i][j]=-1
                Matrix[j][i]=-1
    return Matrix

def Random_bi(L,  P,  Inh):
    matrix=np.zeros((L*L,L*L),int)
    rng=np.random.default_rng(seed=1)
    for i in range(L*L):
        for j in range(L*L):
            I=rng.uniform(0.0,1.0)
            if I<Inh:
                I=0
            else:
                I=1
            if (matrix[i][j]!=1 or matrix[i][j]!=-1) and rng.uniform(0.0,1.0)<P:
            
                matrix[i][j]=-1*(2*I-1)
                matrix[j][i]=-1*(2*I-1)
    return matrix   
    

In [6]:
low_cutoff = 0.01  # Low cut-off frequency (Hz)
high_cutoff = 80.0  # High cut-off frequency (Hz)
order = 4  # Order of the filter
sampling_rate = 500 
b, a = signal.butter(order, [low_cutoff / (0.5 * sampling_rate), high_cutoff / (0.5 * sampling_rate)], btype='band')

fs = 200  # Sampling rate
nperseg = int(fs * 4)  # Segment length
noverlap = int(fs * 2)  # Overlap between segments


In [7]:

def Simulado_EGG(params):
    L, P, inhibidoras, trest, trelative, alpha, tmax, type_matrix = params
    # Compilación del programa
    resultado_compilacion = subprocess.run("g++ -O3 main.cpp Matrix_conect.cpp -o main", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if resultado_compilacion.returncode != 0:
        print("Error durante la compilación:")
        print(resultado_compilacion.stderr)
        return None
    
    # Ejecución del programa
    resultado_ejecucion = subprocess.run(f"./main {L} {P} {inhibidoras} {trest} {trelative} {alpha} {tmax} {type_matrix}", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if resultado_ejecucion.returncode != 0:
        print("Error durante la ejecución:")
        print(resultado_ejecucion.stderr)
        return None
    
    # Procesamiento de los datos
    lines = resultado_ejecucion.stdout.strip().split('\n')
    data_list = [[float(num) for num in line.split()] for line in lines]
    data = np.array(data_list)[:,0]
    
    butter = signal.filtfilt(b, a, data[30:])
    frequencies, psd = signal.welch(butter, fs, nperseg=nperseg, noverlap=noverlap)
    frequencies=np.array(frequencies)
    welch_data = np.array([10 * np.log10(psd[i]**2) for i in range(len(frequencies))])
    
    delta = welch_data[np.logical_and(frequencies >= 1, frequencies <= 3)]
    theta = welch_data[np.logical_and(frequencies >= 4, frequencies <= 7)]
    alpha = welch_data[np.logical_and(frequencies >= 7, frequencies <= 13)]
    beta = welch_data[np.logical_and(frequencies >= 14, frequencies <= 30)]

    return {
        'data': data,"butter": butter,"welch_data":welch,
        'delta':[delta.mean(),delta.std()],
        'theta':[theta.mean(), theta.std()],
        'alpha':[ alpha.mean(),alpha.std()],
        'beta': [beta.mean(), beta.std()]
    }



In [12]:
def Datos_reales(nombre):
    data = np.loadtxt(nombre)*1e6
    frequencies, psd = signal.welch(data, fs, nperseg=nperseg, noverlap=noverlap)
    frequencies=np.array(frequencies)
    welch_data = np.array([10 * np.log10(psd[i]**2) for i in range(len(frequencies))])
    
    delta = welch_data[np.logical_and(frequencies >= 1, frequencies <= 3)]
    theta = welch_data[np.logical_and(frequencies >= 4, frequencies <= 7)]
    alpha = welch_data[np.logical_and(frequencies >= 7, frequencies <= 13)]
    beta = welch_data[np.logical_and(frequencies >= 14, frequencies <= 30)]

    return {
        'data': data,"butter": butter,"welch_data":welch,
        'delta':[delta.mean(),delta.std()],
        'theta':[theta.mean(), theta.std()],
        'alpha':[ alpha.mean(),alpha.std()],
        'beta': [beta.mean(), beta.std()]
    }


In [13]:
initial_params = [40, 0.01, 0.5, 1, 2, 0.1, 1000, 1]

generated_data = Simulado_EGG(initial_params)
reales_total= R = Datos_reales("F7_MNE.txt")


In [14]:
reales=reales_total["delta"]+reales_total["theta"]+reales_total["alpha"]+reales_total["beta"]

In [15]:
import optuna
import numpy as np  # Assuming np is needed for array operations
import optuna.visualization as vis
import plotly

reales=reales_total["delta"]+reales_total["theta"]+reales_total["alpha"]+reales_total["beta"]

def objective(trial):
    params = np.zeros(8)
    
    # Suggesting parameters through the trial object
    params[0] = trial.suggest_int('param1', 30, 60)
    params[1] = trial.suggest_float('param2', 0.0, 1.0)
    params[2] = trial.suggest_float('param3', 0.0, 1.0)
    params[3] = trial.suggest_float('param4', -10.0, 15.0)
    # Removed dynamic starting value for param5
    params[4] = trial.suggest_float('param5', -10.0, 20.0)  # Example range, adjust as needed
    params[5] = trial.suggest_float('param6', 0, 1.0)
    # Assuming these are constant and don't change across trials
    params[6] = 1000
    params[7] = 2
    
    generated_data_total = Simulado_EGG(params)
    generated_data = generated_data_total["delta"] + generated_data_total["theta"] + generated_data_total["alpha"] + generated_data_total["beta"]

    # Calculating the difference between generated and real data
    difference = sum(abs(gd - rd) for gd, rd in zip(generated_data, reales))
    return difference

# Create and optimize the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=4)

# Visualizing the optimization history and parameter importance
vis.plot_optimization_history(study)
vis.plot_param_importances(study)

# Printing the best parameters and result
print("Best parameters:", study.best_params)
print("Result:", study.best_value)


[I 2024-07-11 22:43:11,279] A new study created in memory with name: no-name-db4112c9-8b61-46cd-8239-ab2e79615513
[I 2024-07-11 22:43:34,369] Trial 0 finished with value: 294.34257860780156 and parameters: {'param1': 51, 'param2': 0.7430764342756185, 'param3': 0.7921099830875985, 'param4': -3.6810541542136974, 'param5': 0.15922680819063828, 'param6': 0.47054896748688024}. Best is trial 0 with value: 294.34257860780156.


In [ ]:
import plotly

vis.plot_optimization_history(study)
vis.plot_param_importances(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.